# soft voting
- randomforest
- xgbm
- lgbm
- weight 1,1,1




In [2]:
! pip install  scikit-learn pandas numpy matplotlib seaborn Optuna xgboost
# pycaret



In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score, accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns


# data import
df=pd.read_csv('/Users/summerlee/fisa/09_ML/hy-0920-ml/bank.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


## basic soft_voting
result
- Accuracy: 0.7421
- AUC: 0.8000
- Recall: 0.6261


In [16]:


# 데이터 준비 (예시 데이터를 df라고 가정)
X = df.drop(['deposit', 'duration'], axis=1)
df['deposit'] = df['deposit'].replace({'yes': 1, 'no': 0})
y = df['deposit']


# 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 데이터 분할
# 데이터를 train+validation과 test로 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train+validation을 다시 train과 validation으로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)





# 모델 생성
rf = RandomForestClassifier(n_estimators=100, random_state=42)
# gbm = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, random_state=42)
lgbm = LGBMClassifier(n_estimators=100, random_state=42)

# Voting Classifier 생성
voting_clf = VotingClassifier([
    ('rf', rf),
    ('xgb', xgb),
    ('lgbm', lgbm)
], voting='soft', weights=[1, 1, 1])

# 모델 학습
voting_clf.fit(X_train, y_train)

# 예측
y_pred = voting_clf.predict(X_test)
y_pred_proba = voting_clf.predict_proba(X_test)[:, 1]

# accuracy, AUC와 Recall 계산
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred, pos_label=1)

print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Recall: {recall:.4f}")


# ## 특성 중요도 (Random Forest 기준)
# importances = voting_clf.named_estimators_['rf'].feature_importances_
# indices = np.argsort(importances)[::-1]
# feature_names = X.columns

# plt.figure(figsize=(10, 6))
# plt.title("Feature Importances")
# plt.bar(range(X.shape[1]), importances[indices])
# plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=90)
# plt.tight_layout()
# plt.show()

/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/4060499925.py:3: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000858 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452
AUC: 0.8000
Recall: 0.6261
Accuracy: 0.7421


## gridsearch 사용

result

- Accuracy: 0.7425
- AUC: 0.8073
- Recall: 0.6289

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score, make_scorer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import early_stopping


# 데이터 준비
X = df.drop(['deposit', 'duration'], axis=1)
df['deposit'] = df['deposit'].replace({'yes': 1, 'no': 0})
y = df['deposit']

# 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 데이터를 train+validation과 test로 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train+validation을 다시 train과 validation으로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# 커스텀 스코어 함수 (AUC와 Recall의 조화 평균)
def custom_score(y_true, y_pred_proba):
    auc = roc_auc_score(y_true, y_pred_proba)
    recall = recall_score(y_true, y_pred_proba > 0.5, pos_label=1)
    return 2 * (auc * recall) / (auc + recall)

custom_scorer = make_scorer(custom_score, needs_proba=True)

# 각 모델에 대한 파라미터 그리드 정의
param_grids = {
    'rf': {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    'xgb': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7]
    },
    'lgbm': {
        'learning_rate': [0.01, 0.1],
        'num_leaves': [31, 63]
    }
}
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score, make_scorer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping


best_models = {}

for name, model in [('rf', RandomForestClassifier(random_state=42)),
                    ('xgb', XGBClassifier(random_state=42)),
                    ('lgbm', LGBMClassifier(random_state=42))]:
    print(f"Performing GridSearch for {name}...")

    if name == 'lgbm':
        param_grids[name]['n_estimators'] = [1000]  # Set a high number of estimators
        # Use GridSearchCV for parameter tuning
        grid_search = GridSearchCV(model, param_grids[name], cv=3, scoring=custom_scorer, n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Extract the best model and perform early stopping separately
        best_lgbm_model = grid_search.best_estimator_
        best_lgbm_model.fit(X_train, y_train,
                            eval_set=[(X_val, y_val)],
                            eval_metric='auc',
                            callbacks=[early_stopping(50)])

        best_models[name] = best_lgbm_model
    else:
        grid_search = GridSearchCV(model, param_grids[name], cv=3, scoring=custom_scorer, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_models[name] = grid_search.best_estimator_

    print(f"Best parameters for {name}: {grid_search.best_params_}")
    print(f"Best score for {name}: {grid_search.best_score_:.4f}")




# Voting Classifier 생성
voting_clf = VotingClassifier([
    ('rf', best_models['rf']),
    ('xgb', best_models['xgb']),
    ('lgbm', best_models['lgbm'])
], voting='soft', weights=[1, 1, 1])

# 모델 학습 (train+validation 데이터 사용)
voting_clf.fit(X_train_val, y_train_val)

# 예측
y_pred = voting_clf.predict(X_test)
y_pred_proba = voting_clf.predict_proba(X_test)[:, 1]



# accuracy, AUC와 Recall 계산
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred, pos_label=1)

print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Recall: {recall:.4f}")



Performing GridSearch for rf...


/Users/summerlee/.pyenv/versions/3.11.10/envs/py_3_11_ml/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning:

The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.



Best parameters for rf: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
Best score for rf: 0.6974
Performing GridSearch for xgb...
Best parameters for xgb: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best score for xgb: 0.6872
Performing GridSearch for lgbm...
[LightGBM] [Info] Number of positive: 2241, number of negative: 2521
[LightGBM] [Info] Number of positive: 2241, number of negative: 2521
[LightGBM] [Info] Number of positive: 2242, number of negative: 2520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002281 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 722
[LightGBM] [Info] Number of data points in the train set: 4762, number of used features: 41
[LightGBM] [Info] Number of positive: 2242, number of negative: 2520
[LightGBM] [Info] Number of positive: 2241, number of negative: 2521
[LightGBM]

## optuna 사용

- optuna Accuracy: 0.7273
- optuna AUC: 0.7929
- optuna Recall: 0.6401

In [18]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score, make_scorer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from optuna.integration import LightGBMPruningCallback
import joblib

# 데이터 준비 (이전 코드와 동일)
X = df.drop(['deposit', 'duration'], axis=1)
df['deposit'] = df['deposit'].replace({'yes': 1, 'no': 0})
y = df['deposit']

# 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 데이터를 train+validation과 test로 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train+validation을 다시 train과 validation으로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)




# SQLite 데이터베이스 URL 설정
STUDY_NAME = 'bank_marketing_optimization'
DB_URL = f'sqlite:///{STUDY_NAME}.db'


# def custom_score(y_true, y_pred_proba):
#     auc = roc_auc_score(y_true, y_pred_proba)
#     recall = recall_score(y_true, y_pred_proba > 0.5, pos_label=1)
#     return 2 * (auc * recall) / (auc + recall)

def objective(trial):
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 50, 300),
        'max_depth': trial.suggest_int('rf_max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 10)
    }
    
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 1.0),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10)
    }
    
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 1e-3, 1.0),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 20, 100)
    }
    
    rf = RandomForestClassifier(**rf_params, random_state=42)
    xgb = XGBClassifier(**xgb_params, random_state=42)
    lgbm = LGBMClassifier(**lgbm_params, random_state=42)
    
    voting_clf = VotingClassifier([
        ('rf', rf),
        ('xgb', xgb),
        ('lgbm', lgbm)
    ], voting='soft')
    
    voting_clf.fit(X_train, y_train)
    y_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
    
    return custom_score(y_val, y_pred_proba)

# Optuna 스터디 생성 (SQLite 데이터베이스 사용)
study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=DB_URL,
    load_if_exists=True,
    direction='maximize'
)

# 최적화 실행
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터로 모델 생성
best_params = study.best_params
rf = RandomForestClassifier(
    n_estimators=best_params['rf_n_estimators'],
    max_depth=best_params['rf_max_depth'],
    min_samples_split=best_params['rf_min_samples_split'],
    random_state=42
)
xgb = XGBClassifier(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    random_state=42
)
lgbm = LGBMClassifier(
    n_estimators=best_params['lgbm_n_estimators'],
    learning_rate=best_params['lgbm_learning_rate'],
    num_leaves=best_params['lgbm_num_leaves'],
    random_state=42
)

best_voting_clf = VotingClassifier([
    ('rf', rf),
    ('xgb', xgb),
    ('lgbm', lgbm)
], voting='soft')
 
# 최종 모델 학습
best_voting_clf.fit(X_train_val, y_train_val)

# 모델 저장
joblib.dump(best_voting_clf, 'best_voting_clf.joblib')

# 테스트 세트에서 성능 평가
y_pred = best_voting_clf.predict(X_test)
y_pred_proba = best_voting_clf.predict_proba(X_test)[:, 1]

# accuracy, AUC와 Recall 계산
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred, pos_label=1)

print(f"optuna Accuracy: {accuracy:.4f}")
print(f"optuna AUC: {auc:.4f}")
print(f"optuna Recall: {recall:.4f}")


[I 2024-09-20 04:04:10,737] Using an existing study with name 'bank_marketing_optimization' instead of creating a new one.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:48: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:54: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000875 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 04:04:15,905] Trial 200 finished with value: 0.7035383975476716 and parameters: {'rf_n_estimators': 282, 'rf_max_depth': 25, 'rf_min_samples_split': 8, 'xgb_n_estimators': 231, 'xgb_learning_rate': 0.06943489506054942, 'xgb_max_depth': 9, 'lgbm_n_estimators': 272, 'lgbm_learning_rate': 0.16342091280021973, 'lgbm_num_leaves': 41}. Best is trial 40 with value: 0.7109753023947039.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:48: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:54: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000883 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 04:04:21,080] Trial 201 finished with value: 0.6996787067223433 and parameters: {'rf_n_estimators': 285, 'rf_max_depth': 24, 'rf_min_samples_split': 8, 'xgb_n_estimators': 254, 'xgb_learning_rate': 0.0931421944974977, 'xgb_max_depth': 9, 'lgbm_n_estimators': 258, 'lgbm_learning_rate': 0.4363370871563635, 'lgbm_num_leaves': 45}. Best is trial 40 with value: 0.7109753023947039.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:48: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:54: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



...

[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000876 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 04:11:05,872] Trial 298 finished with value: 0.7043544183222211 and parameters: {'rf_n_estimators': 261, 'rf_max_depth': 22, 'rf_min_samples_split': 7, 'xgb_n_estimators': 232, 'xgb_learning_rate': 0.06524930830975542, 'xgb_max_depth': 9, 'lgbm_n_estimators': 264, 'lgbm_learning_rate': 0.18626931436901653, 'lgbm_num_leaves': 46}. Best is trial 257 with value: 0.7118772960901607.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:48: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/847976484.py:54: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 04:11:10,127] Trial 299 finished with value: 0.6997131211222334 and parameters: {'rf_n_estimators': 290, 'rf_max_depth': 24, 'rf_min_samples_split': 8, 'xgb_n_estimators': 224, 'xgb_learning_rate': 0.11559821235262113, 'xgb_max_depth': 9, 'lgbm_n_estimators': 248, 'lgbm_learning_rate': 0.06279629106035517, 'lgbm_num_leaves': 29}. Best is trial 257 with value: 0.7118772960901607.


[LightGBM] [Info] Number of positive: 4222, number of negative: 4707
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000865 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 735
[LightGBM] [Info] Number of data points in the train set: 8929, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.472841 -> initscore=-0.108742
[LightGBM] [Info] Start training from score -0.108742
optuna Accuracy: 0.7273
optuna AUC: 0.7929
optuna Recall: 0.6401


In [7]:

# 커스텀 스코어 함수 (AUC와 Recall의 조화 평균)
def custom_score(y_true, y_pred_proba):
    auc = roc_auc_score(y_true, y_pred_proba)
    recall = recall_score(y_true, y_pred_proba > 0.5, pos_label=1)
    return 2 * (auc * recall) / (auc + recall)

# Optuna를 사용한 하이퍼파라미터 최적화
def objective(trial):
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 50, 300),
        'max_depth': trial.suggest_int('rf_max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 10)
    }
    
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 1.0),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10)
    }
    
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 1e-3, 1.0),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 20, 100)
    }
    
    rf = RandomForestClassifier(**rf_params, random_state=42)
    xgb = XGBClassifier(**xgb_params, random_state=42)
    lgbm = LGBMClassifier(**lgbm_params, random_state=42)
    
    voting_clf = VotingClassifier([
        ('rf', rf),
        ('xgb', xgb),
        ('lgbm', lgbm)
    ], voting='soft')
    
    voting_clf.fit(X_train, y_train)
    y_pred_proba = voting_clf.predict_proba(X_val)[:, 1]
    
    return custom_score(y_val, y_pred_proba)

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize', study_name='bank_marketing_optimization')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터로 모델 생성
best_params = study.best_params
rf = RandomForestClassifier(
    n_estimators=best_params['rf_n_estimators'],
    max_depth=best_params['rf_max_depth'],
    min_samples_split=best_params['rf_min_samples_split'],
    random_state=42
)
xgb = XGBClassifier(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    random_state=42
)
lgbm = LGBMClassifier(
    n_estimators=best_params['lgbm_n_estimators'],
    learning_rate=best_params['lgbm_learning_rate'],
    num_leaves=best_params['lgbm_num_leaves'],
    random_state=42
)

best_voting_clf = VotingClassifier([
    ('rf', rf),
    ('xgb', xgb),
    ('lgbm', lgbm)
], voting='soft')

# 최종 모델 학습
best_voting_clf.fit(X_train_val, y_train_val)

# 모델 저장
joblib.dump(best_voting_clf, 'best_voting_clf.joblib')

# 테스트 세트에서 성능 평가
y_pred = best_voting_clf.predict(X_test)
y_pred_proba = best_voting_clf.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred, pos_label=1)
custom_score_value = custom_score(y_test, y_pred_proba)



[I 2024-09-20 03:00:29,632] A new study created in memory with name: bank_marketing_optimization
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 1.0),
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 1e-3, 1.0),


[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 03:00:31,875] Trial 0 finished with value: 0.6865007965959817 and parameters: {'rf_n_estimators': 137, 'rf_max_depth': 10, 'rf_min_samples_split': 7, 'xgb_n_estimators': 105, 'xgb_learning_rate': 0.0010858672397783306, 'xgb_max_depth': 3, 'lgbm_n_estimators': 173, 'lgbm_learning_rate': 0.3098067994913723, 'lgbm_num_leaves': 60}. Best is trial 0 with value: 0.6865007965959817.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 1.0),
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use 

[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001035 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 03:00:36,326] Trial 1 finished with value: 0.687827754775586 and parameters: {'rf_n_estimators': 274, 'rf_max_depth': 17, 'rf_min_samples_split': 7, 'xgb_n_estimators': 113, 'xgb_learning_rate': 0.08193602485634323, 'xgb_max_depth': 7, 'lgbm_n_estimators': 267, 'lgbm_learning_rate': 0.02103318755258792, 'lgbm_num_leaves': 59}. Best is trial 1 with value: 0.687827754775586.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 1.0),
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use sug

...

[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000920 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 03:06:09,265] Trial 98 finished with value: 0.7024198341873777 and parameters: {'rf_n_estimators': 221, 'rf_max_depth': 24, 'rf_min_samples_split': 9, 'xgb_n_estimators': 86, 'xgb_learning_rate': 0.10337094490919163, 'xgb_max_depth': 9, 'lgbm_n_estimators': 250, 'lgbm_learning_rate': 0.0898139637746031, 'lgbm_num_leaves': 65}. Best is trial 82 with value: 0.7092890820329893.
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 1.0),
/var/folders/0h/6t5rrrkd413d6v0z43zmchmm0000gn/T/ipykernel_38998/738175.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use s

[LightGBM] [Info] Number of positive: 3362, number of negative: 3781
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 7143, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470671 -> initscore=-0.117452
[LightGBM] [Info] Start training from score -0.117452


[I 2024-09-20 03:06:13,935] Trial 99 finished with value: 0.6921957408645932 and parameters: {'rf_n_estimators': 228, 'rf_max_depth': 27, 'rf_min_samples_split': 10, 'xgb_n_estimators': 276, 'xgb_learning_rate': 0.43883812269882744, 'xgb_max_depth': 10, 'lgbm_n_estimators': 274, 'lgbm_learning_rate': 0.15591303972707565, 'lgbm_num_leaves': 51}. Best is trial 82 with value: 0.7092890820329893.


[LightGBM] [Info] Number of positive: 4222, number of negative: 4707
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000994 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 735
[LightGBM] [Info] Number of data points in the train set: 8929, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.472841 -> initscore=-0.108742
[LightGBM] [Info] Start training from score -0.108742
AUC: 0.7984
Recall: 0.6457
Custom Score: 0.7140


In [14]:
from sklearn.metrics import roc_auc_score, recall_score, accuracy_score, precision_score, f1_score, make_scorer

# 테스트 세트에서 성능 평가
y_pred = best_voting_clf.predict(X_test)
y_pred_proba = best_voting_clf.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
custom_score_value = custom_score(y_test, y_pred_proba)


print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Custom Score: {custom_score_value:.4f}")

# Optuna 최적화 결과 출력
print(f"Best trial value (from Optuna): {study.best_value:.4f}")
print("Best hyperparameters:", study.best_params)

Accuracy: 0.7371
AUC: 0.7995
Recall: 0.6429
Precision: 0.7691
F1 Score: 0.7004
Custom Score: 0.7127
Best trial value (from Optuna): 0.7110
Best hyperparameters: {'rf_n_estimators': 176, 'rf_max_depth': 20, 'rf_min_samples_split': 9, 'xgb_n_estimators': 162, 'xgb_learning_rate': 0.084437396689471, 'xgb_max_depth': 5, 'lgbm_n_estimators': 270, 'lgbm_learning_rate': 0.39502413530407376, 'lgbm_num_leaves': 46}
